In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster


In [2]:
# URL of NSW Govt Case Locations
URL = 'https://www.health.nsw.gov.au/Infectious/covid-19/Pages/case-locations-and-alerts.aspx'
page = requests.get(URL)

In [3]:
# Use Beautiful Soup to clean up HTML
soup = BeautifulSoup(page.content,'html.parser')
table = soup.find_all('table')[1] # Grab the second table

data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])


# Drop First and Last rows
data = data[1:-1]

# Create dataframe
df = pd.DataFrame(np.array(data), columns=['Suburb','Venue','Address','Date and time','Health advice'])

In [4]:
# Drop unnecessary column
df = df.drop(['Health advice'], axis =1 )
df

,Suburb,Venue,Address,Date and time
0,Avalon Beach,Avalon Beach RSL Club,"1 Bowling Green Lane, Avalon Beach, NSW 2107",All day until closing on Friday 11 December
1,Avalon Beach,Avalon Bowlo (Bowling Club),"4 Bowling Green Lane, Avalon Beach, NSW 2107",5pm to 7pm on Sunday 13 December 3pm-5pm on Tu...
2,Avalon Beach,Bangkok Sidewalk Restaurant,"1/21-23 Old Barrenjoey Road, Avalon Beach, NSW...",7pm to 8pm on Monday 14 December
3,Avalon Beach,Barramee Thai Massage and Spa,"4/42-44 Old Barrenjoey Road, Avalon Beach, NSW...",2pm to 3:30pm on Monday 14 December
4,Avalon Beach,Sneaky Grind Café,"3/48 Old Barrenjoey Road, Avalon Beach, NSW 2107",9:30am to 11am on Monday 14 December
5,Cronulla,Cronulla RSL Club,"38R Gerrale Street, Cronulla NSW 2230",5pm to closing on Wednesday 16 December
6,Cronulla,Pilgrims Vegetarian Café,"97 Gerrale St, Cronulla NSW 2230",11:30am to 2:30pm on Wednesday 16 December
7,Kirribilli,Kirribilli Club,"11 Harbourview Crescent, Lavender Bay, NSW 2060",12pm to 3pm on Monday 14 December
8,Lane Cove,Hair by Erika,"The Village Shopping Center, Burns Bay Road, L...",3:30pm to 5pm on Friday 11 December
9,Newport,Rusti Fig,"3/363 Barrenjoey Road, Newport NSW 2106",9am to 10am on Saturday 12 December


In [5]:
# Reference: https://towardsdatascience.com/geocode-with-python-161ec1e62b89

# Use Nominatim as geocoder
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['Address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df = df.dropna()
df

,Suburb,Venue,Address,Date and time,location,point
0,Avalon Beach,Avalon Beach RSL Club,"1 Bowling Green Lane, Avalon Beach, NSW 2107",All day until closing on Friday 11 December,"(1, Bowling Green Lane, Avalon Beach, Sydney, ...","(-33.6354409, 151.3285888, 0.0)"
1,Avalon Beach,Avalon Bowlo (Bowling Club),"4 Bowling Green Lane, Avalon Beach, NSW 2107",5pm to 7pm on Sunday 13 December 3pm-5pm on Tu...,"(Bowling Green Lane, Avalon Beach, Sydney, Nor...","(-33.6351897, 151.3282649, 0.0)"
2,Avalon Beach,Bangkok Sidewalk Restaurant,"1/21-23 Old Barrenjoey Road, Avalon Beach, NSW...",7pm to 8pm on Monday 14 December,"(21-23, Old Barrenjoey Road, Avalon Beach, Syd...","(-33.6374524, 151.3282604, 0.0)"
3,Avalon Beach,Barramee Thai Massage and Spa,"4/42-44 Old Barrenjoey Road, Avalon Beach, NSW...",2pm to 3:30pm on Monday 14 December,"(42-44, Old Barrenjoey Road, Avalon Beach, Syd...","(-33.6379585, 151.3288214, 0.0)"
4,Avalon Beach,Sneaky Grind Café,"3/48 Old Barrenjoey Road, Avalon Beach, NSW 2107",9:30am to 11am on Monday 14 December,"(Old Barrenjoey Road, Avalon Beach, Sydney, No...","(-33.6347755, 151.3305118, 0.0)"
5,Cronulla,Cronulla RSL Club,"38R Gerrale Street, Cronulla NSW 2230",5pm to closing on Wednesday 16 December,"(Gerrale Street, Cronulla, Sydney, Sutherland ...","(-34.0517095, 151.1544662, 0.0)"
6,Cronulla,Pilgrims Vegetarian Café,"97 Gerrale St, Cronulla NSW 2230",11:30am to 2:30pm on Wednesday 16 December,"(Gerrale Street, Cronulla, Sydney, Sutherland ...","(-34.0517095, 151.1544662, 0.0)"
7,Kirribilli,Kirribilli Club,"11 Harbourview Crescent, Lavender Bay, NSW 2060",12pm to 3pm on Monday 14 December,"(Harbourview Crescent, Lavender Bay, Sydney, N...","(-33.8437492, 151.2093509, 0.0)"
8,Lane Cove,Hair by Erika,"The Village Shopping Center, Burns Bay Road, L...",3:30pm to 5pm on Friday 11 December,"(Coxs Road Mall, Coxs Road, North Ryde, Sydney...","(-33.79424265, 151.1222521703526, 0.0)"
9,Newport,Rusti Fig,"3/363 Barrenjoey Road, Newport NSW 2106",9am to 10am on Saturday 12 December,"(Barrenjoey Road, Newport, Sydney, Northern Be...","(-33.6565111, 151.3186419, 0.0)"


In [6]:
# Convert points to lat, lon, alt
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

# Create map
map1 = folium.Map(
    location=[-33.5992903, 151.3251659],
    tiles='cartodbpositron',
    zoom_start=12,
)

# Add points to map
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
df

,Suburb,Venue,Address,Date and time,location,point,latitude,longitude,altitude
0,Avalon Beach,Avalon Beach RSL Club,"1 Bowling Green Lane, Avalon Beach, NSW 2107",All day until closing on Friday 11 December,"(1, Bowling Green Lane, Avalon Beach, Sydney, ...","(-33.6354409, 151.3285888, 0.0)",-33.635441,151.328589,0.0
1,Avalon Beach,Avalon Bowlo (Bowling Club),"4 Bowling Green Lane, Avalon Beach, NSW 2107",5pm to 7pm on Sunday 13 December 3pm-5pm on Tu...,"(Bowling Green Lane, Avalon Beach, Sydney, Nor...","(-33.6351897, 151.3282649, 0.0)",-33.635190,151.328265,0.0
2,Avalon Beach,Bangkok Sidewalk Restaurant,"1/21-23 Old Barrenjoey Road, Avalon Beach, NSW...",7pm to 8pm on Monday 14 December,"(21-23, Old Barrenjoey Road, Avalon Beach, Syd...","(-33.6374524, 151.3282604, 0.0)",-33.637452,151.328260,0.0
3,Avalon Beach,Barramee Thai Massage and Spa,"4/42-44 Old Barrenjoey Road, Avalon Beach, NSW...",2pm to 3:30pm on Monday 14 December,"(42-44, Old Barrenjoey Road, Avalon Beach, Syd...","(-33.6379585, 151.3288214, 0.0)",-33.637959,151.328821,0.0
4,Avalon Beach,Sneaky Grind Café,"3/48 Old Barrenjoey Road, Avalon Beach, NSW 2107",9:30am to 11am on Monday 14 December,"(Old Barrenjoey Road, Avalon Beach, Sydney, No...","(-33.6347755, 151.3305118, 0.0)",-33.634776,151.330512,0.0
5,Cronulla,Cronulla RSL Club,"38R Gerrale Street, Cronulla NSW 2230",5pm to closing on Wednesday 16 December,"(Gerrale Street, Cronulla, Sydney, Sutherland ...","(-34.0517095, 151.1544662, 0.0)",-34.051710,151.154466,0.0
6,Cronulla,Pilgrims Vegetarian Café,"97 Gerrale St, Cronulla NSW 2230",11:30am to 2:30pm on Wednesday 16 December,"(Gerrale Street, Cronulla, Sydney, Sutherland ...","(-34.0517095, 151.1544662, 0.0)",-34.051710,151.154466,0.0
7,Kirribilli,Kirribilli Club,"11 Harbourview Crescent, Lavender Bay, NSW 2060",12pm to 3pm on Monday 14 December,"(Harbourview Crescent, Lavender Bay, Sydney, N...","(-33.8437492, 151.2093509, 0.0)",-33.843749,151.209351,0.0
8,Lane Cove,Hair by Erika,"The Village Shopping Center, Burns Bay Road, L...",3:30pm to 5pm on Friday 11 December,"(Coxs Road Mall, Coxs Road, North Ryde, Sydney...","(-33.79424265, 151.1222521703526, 0.0)",-33.794243,151.122252,0.0
9,Newport,Rusti Fig,"3/363 Barrenjoey Road, Newport NSW 2106",9am to 10am on Saturday 12 December,"(Barrenjoey Road, Newport, Sydney, Northern Be...","(-33.6565111, 151.3186419, 0.0)",-33.656511,151.318642,0.0
